# Tutorial to get the initial values for the mixing matrix B in different cases
(PySm models, customized models, w or w/o spv in the models, in the comp sep patches)

In [3]:
import numpy as np
import healpy as hp
from micmac.third_party import get_instrument, standardize_instrument

from micmac.templates_spv import create_one_template, get_n_patches_b, tree_spv_config, get_nodes_b
from micmac.customized_models import fgs_freq_maps_from_customized_model_nonparam, parametric_sky_customized, get_spectral_params_true_values
from micmac.init_mixingmatrix import InitMixingMatrix

In [4]:
##### Code validation
instr = get_instrument("LiteBIRD")
instr = standardize_instrument(instr)
nside_maps = 64
ncomp = 3
n_fgs_comp = ncomp - 1
pos_special_freqs = np.array([0, -1])
# Num betas (= number of params in B if it was pixel indep)
n_betas = (ncomp-1)*(len(instr.frequency)-ncomp+1)
# Read the spv configuration and print the spv tree
yaml_file_path = 'params_spv_LB.yaml'
root_tree = tree_spv_config(yaml_file_path, n_betas, n_fgs_comp, print_tree=False)
# Get list of the nodes betas
nodes_b = get_nodes_b(root_tree)
# Get the nside of the spv for customized model
nside_spv_model = 2

<_io.TextIOWrapper name='params_spv_LB.yaml' mode='r' encoding='UTF-8'>
count_b: 26
n_betas:  26


In [13]:
### 1) With default (from spectral params and s0 d0)
init_mixing_matrix1 = InitMixingMatrix(instr.frequency, 
                                        ncomp, 
                                        pos_special_freqs, 
                                        spv_nodes_b=nodes_b,
                                        nside=nside_maps)
# fgs_SEDs1 = init_mixing_matrix1.fgs_SEDs_from_spectral_params()
# params1 = init_mixing_matrix1.init_params_from_fgs_SEDs(fgs_SEDs1)
# print(params1.shape)
init_params1 = init_mixing_matrix1.init_params()
print(init_params1.shape)
print(init_params1)

>>> init params built with spectral params: 1.54 20.0 -3.0
(26,)
[0.52387813 0.3117258  0.21971687 0.15096497 0.10631274 0.07881344
 0.05159725 0.03585547 0.02552178 0.0193825  0.01493871 0.01211497
 0.00871575 0.00182426 0.00324908 0.00433627 0.0057459  0.0074421
 0.00935487 0.0133358  0.01906532 0.02888587 0.04508262 0.08217483
 0.16061503 0.37616284]


In [6]:
### 2) Passing arbitrary spectral params
### This is not really a case on which we can apply the init_params method
### because for that we need either one parameter per pixel or one on the whole sky
init_mixing_matrix2 = InitMixingMatrix(instr.frequency, 
                                        ncomp, 
                                        pos_special_freqs, 
                                        spv_nodes_b=nodes_b, 
                                        beta_mbb=[1.54,1.54,1.54,1.54], 
                                        temp_mbb=[20.0,20.0,20.0,20.0], 
                                        beta_pl=[-3.0,-3.0,-3.0,-3.0],
                                        nside=nside_maps)
fgs_SEDs2 = init_mixing_matrix2.fgs_SEDs_from_spectral_params()
params2 = init_mixing_matrix2.init_params_from_fgs_SEDs(fgs_SEDs2)
print(params2.shape)

(15, 2, 4)


In [7]:
### 3) Passing spectral params from d1 s1
B_s, B_d, T_d = get_spectral_params_true_values(nside_maps, model=['s1', 'd1'])
init_mixing_matrix3 = InitMixingMatrix(instr.frequency, 
                                        ncomp, 
                                        pos_special_freqs, 
                                        spv_nodes_b=nodes_b, 
                                        beta_pl=B_s,
                                        beta_mbb=B_d,
                                        temp_mbb=T_d,
                                        nside=nside_maps)
fgs_SEDs3 = init_mixing_matrix3.fgs_SEDs_from_spectral_params()
params3 = init_mixing_matrix3.init_params_from_fgs_SEDs(fgs_SEDs3)    
print(params3.shape)
init_params3 = init_mixing_matrix3.init_params()
print(init_params3.shape)

(15, 2, 49152)
>>> init params built with spectral params: [1.5431979 1.5527593 1.5420678 ... 1.5674531 1.55847   1.561273 ] [23.088667 23.690893 23.575125 ... 24.509298 23.702038 23.98124 ] [-3.0075154 -3.00687   -3.00875   ... -3.016255  -3.011593  -3.0155716]
(26,)


In [8]:
### 4) Passing sp param values as we get them from a customized model
### still parametric but with less spv of the spectral parameters
_, sp_params = parametric_sky_customized(['s1', 'd1'], nside_maps, nside_spv_model)
init_mixing_matrix4 = InitMixingMatrix(instr.frequency, 
                                        ncomp, 
                                        pos_special_freqs, 
                                        spv_nodes_b=nodes_b, 
                                        beta_pl=sp_params[0],
                                        beta_mbb=sp_params[1],
                                        temp_mbb=sp_params[2],
                                        nside=nside_maps)
fgs_SEDs4 = init_mixing_matrix4.fgs_SEDs_from_spectral_params()
params4 = init_mixing_matrix4.init_params_from_fgs_SEDs(fgs_SEDs4)
print(params4.shape)
init_params4 = init_mixing_matrix4.init_params()
print(init_params4.shape)

(15, 2, 49152)
>>> init params built with spectral params: [1.5530994 1.5556521 1.5302141 ... 1.5400765 1.5583149 1.5564084] [23.587074 23.652555 21.745665 ... 22.944717 23.839792 23.664848] [-3.007767  -3.0000684 -2.983066  ... -2.977777  -2.9681702 -3.0613744]
(26,)


In [9]:
### 5) Passing mixing matrix from a customized model (mixing matrix A built from freq maps)
### with s0 and d0 to check that the customized model is correct 
### (it must return the same as when built from spectral params)
_, custom_fgs_SED = fgs_freq_maps_from_customized_model_nonparam(nside_maps, 
                                                                nside_spv=nside_spv_model, 
                                                                instrument=instr, 
                                                                fgs_models=['s0','d0'], 
                                                                idx_ref_freq=0,
                                                                return_mixing_mat=True)
init_mixing_matrix5 = InitMixingMatrix(instr.frequency, 
                                        ncomp, 
                                        pos_special_freqs, 
                                        spv_nodes_b=nodes_b, 
                                        non_param_fgs_mixing_matrix=custom_fgs_SED,
                                        nside=nside_maps)
params5 = init_mixing_matrix5.init_params_from_fgs_SEDs(custom_fgs_SED)    
print(params5.shape)
init_params5 = init_mixing_matrix5.init_params()
print(init_params5.shape)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


(15, 2, 49152)
>>> init params built from fgs mixing matrix (A) passed by the user:
(26,)


In [14]:
### 6) Passing mixing matrix from a customized model (mixing matrix A built from freq maps)
_, custom_fgs_SED = fgs_freq_maps_from_customized_model_nonparam(nside_maps, 
                                                                nside_spv=nside_spv_model, 
                                                                instrument=instr, 
                                                                fgs_models=['s1','d7'],
                                                                idx_ref_freq=0,
                                                                return_mixing_mat=True)
init_mixing_matrix6 = InitMixingMatrix(instr.frequency, 
                                        ncomp, 
                                        pos_special_freqs, 
                                        spv_nodes_b=nodes_b, 
                                        non_param_fgs_mixing_matrix=custom_fgs_SED,
                                        nside=nside_maps)
params6 = init_mixing_matrix6.init_params_from_fgs_SEDs(custom_fgs_SED)    
print(params6.shape)
init_params6 = init_mixing_matrix6.init_params()
print(init_params6.shape)
print(init_params6)

(15, 2, 49152)
>>> init params built from fgs mixing matrix (A) passed by the user:
(26,)
[0.52389    0.31174841 0.21974192 0.15098716 0.10632718 0.07881769
 0.05158116 0.03581498 0.02544781 0.01926525 0.01474828 0.0118301
 0.00834465 0.0022972  0.00419783 0.0055749  0.0073112  0.0093673
 0.01165466 0.01633964 0.02294236 0.03405085 0.05202305 0.09224578
 0.17477955 0.39387346]
